In [1]:
!pip install sastrawi

In [2]:
!pip install nltk

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [4]:
#import pandas : membaca file csv yang akan digunakan dalam program
import pandas as pd
files = ['Dataset.csv']
df = pd.read_csv(files[0]) #membaca file csv nya

In [5]:
#untuk mencetak data csv yang digunakan dalam bentuk raw
print(df)

       Shortcode            Username  \
0    B0DvRXBDKFm          vhijayfams   
1    B0DvRXBDKFm         bad_sibad18   
2    B0DvRXBDKFm         lissasyam23   
3    B0DvRXBDKFm  hendra_realdista07   
4    B0DvRXBDKFm    tubagusnurulalam   
..           ...                 ...   
995  B30umVxFPKo     marsya_muhammad   
996  B30umVxFPKo         epok_831611   
997  B30umVxFPKo           andhywhan   
998  B30umVxFPKo            yudhia27   
999  B30umVxFPKo       shintaemrlda_   

                                               Comment  Label  
0    Mantap bang ente tinggi rambut gondrong cocok ...      0  
1    Kan kalo gini Baru cewek kalem lemah lembut ga...      0  
2    Lama lama benci sama lucinta dulunya suka Seka...      1  
3                Jadi bencong kok gitu banget sih mas.      1  
4              Aku mau mengembalikan tititku yg hilang      1  
..                                                 ...    ...  
995                                  Jogednya kaku coy      1  
996  Pa

In [6]:
#Preprocessing
import re #library untuk RegEx (Regular Expression)
import nltk #library untuk tokenizing
import numpy as np #untuk array
from scipy.sparse import csr_matrix #


from nltk.tokenize import word_tokenize 
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory #Stopword
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #Stemming


def clean_word(word, casefolding=True, #true false itu boolean 
               remove_mention=True, 
               remove_url=False,
               tokenization=True,
               remove_slang=True,
               remove_repeat_char=False, 
               remove_stopword=True, 
               stemming=True,
               slangs=True, 
               stopwords=[],
               berulang=[]):
    cleantext = word
    if(casefolding): # if ngejalanin yang true
        cleantext = word.lower()
        
    if(remove_mention):
        cleantext = re.sub('@\S+', '', cleantext) #ngambil @ diikuti yang gada spasi (\S+ : nggada spasinya)
    if(remove_url):
        cleantext = re.sub('http\S+\s*', '', cleantext) #
    
    if(tokenization):
        tokens = nltk.word_tokenize(cleantext)
        cleantext = ' '.join(tokens) #text = #["fitriani", "azizah"]
                                            #nama = ' '.join(text)
                                            #hasil nama = "fitriani azizah"
    
    for slang, formal in slangs.items():
        cleantext = re.sub(r'\b%s\b'%re.escape(slang), formal, cleantext) #\b' (string = re.escape)
    cleantext = re.sub(r'\b(\w*(wk|kwo)\w*)\b','',cleantext) 
    cleantext = re.sub(r'\b(w*(wk|kwa)\w*)\b','',cleantext)
    cleantext = re.sub(r'\b(\w*haha\w*)\b','',cleantext)
    cleantext = re.sub(r'\b(\w*ekek\w*)\b','',cleantext)
    cleantext = re.sub(r'\b(\w*hehe\w*)\b','',cleantext)
    cleantext = re.sub(r'\b(\w*(h|eh)\w*)\b','',cleantext)
        
    if(remove_stopword):
        factory = StopWordRemoverFactory()
        stopwords = factory.get_stop_words() + stopwords
        temp = [t for t in re.findall(r'\b[a-z]+-?[a-z]+\b',cleantext) if t not in stopwords]
        cleantext = ' '.join(temp)      
   
    if(stemming):
        stemmer = StemmerFactory().create_stemmer()
        cleantext = stemmer.stem(cleantext)
        return cleantext #dibalikin ke data aslinya
#print("[%s] ---  selesai melakukan preprocessing" % (no))
    print("result: ")
    print(cleantext)
    
    

In [7]:
with open('id.stopwords.02.01.2016.txt') as f:
    myNames = [line.strip() for line in f]

In [8]:
with open('kata-berulang.txt') as f:
    myNames = [line.strip() for line in f]

In [9]:
docs_clean = []
slangs = {'pdhal':'padahal', 'sm':'sama', 'gtu':'begitu', 'syantik':'cantik', 'syuka':'suka', 'nggak':'tidak', 'ane':'aku', 'ktmu':'ketemu', 'cantiq':'cantik',
          'aq':'aku', 'netijen':'netizen', 'amvun':'ampun', 'op':'operasi', 'syekalih':'sekali', 'kaya':'seperti', 'syukak':'suka', 'dongs':'dong', 'mba':'mbak',
          'lu':'kamu', 'gegara':'gara gara', 'tnpa':'tanpa', 'kl':'kalau', 'bngt':'banget', 'donk':'dong', 'bgtu':'begitu', 'kpd':'kepada',
          'kagak':'tidak', 'gak':'tidak', 'kayak':'seperti', 'cewe':'cewek', 'bacod':'bacot', 'tmbh':'tambah', 'ktm':'ketemu', 'maapkeun':'maaf', 
          'jan':'jangan', 'dri':'dari', 'sdh':'sudah', 'boong':'bohong', 'mehong':'mahal', 'couo':'cowok', 'ketjil':'kecil', 'ente':'kamu', 'qw':'aku', 
          'cowo':'cowok', 'manjaah':'manja', 'u':'kamu', 'drpd':'daripada', 'gt':'begitu', 'nggapapa':'tidak apa', 'tytyd':'titik', 'kaga':'tidak',
          'gtw':'tidak tahu', 'bpk':'bapak', 'w':'aku', 'fose':'pose','bat':'banget', 'bodok':'bodoh', 'makaxa':'makannya',
          'azabx':'azabnya', 'piara':'pelihara', 'ngumplin':'ngumpulin', 'islm':'islam', 'it':'itu', 'sm':'sama', 'emg':'emang', 'tpi':'tapi', 
          'dn':'dan', 'salh':'salah', 'km':'kamu', 'dr':'dari', 'sdh':'sudah', 'jd':'jadi', 'gmn':'bagaimana', 'udeh':'sudah','gmna':'bagaimana',
          'yng':'yang', 'jng':'mbak', 'bnget':'banget', 'ente':'kamu', 'gini':'begini', 'triak':'teriak', 'gak':'tidak', 'ga':'tidak','bgt':'banget',  'ehhh':'', 'nya':'', 'cucok':'cocok', 'akutuh':'aku', 'lu':'kamu',
          'jgn':'jangan',  'gk':'tidak', 'kek':'seperti', 'bnr':'benar', 'yg':'yang', 'LL':'lucinta luna','sygnya':'sayangnya', 'kayak':'seperti', 'luh':'kamu', 'dh':'', 'cowo':'cowok', 'sih':'', 'woy':'',
          'aja':'saja', 'lo':'kamu', 'deh':'', 'org':'orang', 'cma':'cuma', 'krn':'karena', 'preman':'pereman','deh':'', 'jgn':'jangan', 'km':'kamu', 'kok':'', 'yng':'yang', 'cewe':'cewek', 'ku':'aku', 'pdhl':'padahal',
          'is':'', 'cwo':'cowok', 'bgtu':'begitu', 'sampe':'sampai', 'kepengen':'ingin', 'puced':'pucat','yah':'ya', 'ntr':'nanti', 'netijen':'netizen', 'tt':'tetek', 'ketjil':'kecil', 'tp':'tapi', 'klo':'kalau',
          'gtu':'begitu', 'cwe':'cewek', 'atu':'satu', 'tdk':'tidak', 'mbk':'mbak','gw':'aku', 'pict':'gambar', 'udh':'sudah', 'kyk': 'seperti', 'ntar':'nanti','pd':'pada', 'ttg':'tentang','knp':'kenapa', 'aq':'aku', 'gaes':'', 'tersyuyuunkkk':'tersayang',
          'elu':'kamu', 'fose':'pose' , 'blm':'belum',  'brp':'berapa', 'duh':'aduh','mah':'', 'tuh':'itu', 'tu':'itu', 'mehong':'mahal', 'bener':'benar',
          'kok':'', 'gua':'aku', 'lg':'lagi', 'tpi':'tapi', 'liat':'lihat', 'holang':'orang','kl':'kalau', 'ato':'atau', 'eh':'', 'gt':'begitu', 'bgd':'banget', 'ama':'sama', 'laen':'lain',
          'wah':'', 'jamiun':'', 'plis':'', 'trus':'terus', 'sma':'sama', 'sih':'', 'gt':'begitu','jgn':'jangan', 'nich':'nih', 'kyak':'kayak', 'syekalih':'sekali', 'bgni':'begini', 'op':'operasi',
          'syukak':'suka', 'uda':'sudah', 'qq':'kakak', 'ni':'ini', 'tamvan':'tampan', 'cntik':'cantik', 'tu':'itu','dgn':'dengan', 'dh':'', 'ups' :'',  'couo':'cowok', 'kpd':'kepada', 'donk':'dong', 'u':'kamu', 'bkn':'bukan',
          'aer':'air',  'maen':'main', 'dlu':'dulu', 'w':'aku', 'byk':'banyak', 'koq':'kok','blg':'bilang', 'blm':'belum', 'pansos':'panjat sosial', 'hei':'', 'kmu':'kamu', 'anaknx':'anaknya',               'sok':'', 'elo':'kamu', 'nggak':'tidak', 'kagak':'tidak', 'd':'di', 'woy':'', 'ah':'', 'hy':'hai','ko':'kok','syedih':'sedih',
          'cantim':'cantik','bnr':'benar','sygnya':'sayangnya','luh':'kamu','dy':'dia','ka':'kakak','tp':'tapi','luu':'kamu', 
          'tuh':'itu','blm':'belum','kalo':'kalau','begitoh':'begitu','krn':'karena','azabx':'azabnya','gmn':'bagaimana','kepengen':'ingin',
          'cntk':'cantik','kmrn':'kemarin','qq':'kakak','org':'orang','ntr':'nanti','ttg':'tentang','ku':'aku','puced':'pucat','titid':'titit','ituu':'itu',
          'ato':'atau','kyk':'seperti','tabalek':'terbalik','jgn':'jangan','tersyuyuunkkk':'tersayang','tpi':'tapi','km':'kamu','it':'itu', 'syank':'sayang',
          'pake':'pakai','klo':'kalau','bat':'banget','gimana':'bagaimana','mb':'mbak','qaqa':'kakak','gk':'tidak','ni':'ini','holang':'orang', 'ngeliatin':'lihat',
          'krn':'karena','tw':'tahu','gw':'aku','atw':'atau','udeh':'udah','kek':'seperti','liat':'lihat','ttp':'tetap','brp':'berapa','mbk':'mbak','preman':'pereman',
          'lo':'kamu','tu':'itu','bgd':'banget','klw':'kalau','bnget':'banget','pdhal':'padahal','waw':'', 'sm':'sama', 'gtu':'begitu', 'syantik':'cantik', 'syuka':'suka', 'nggak':'tidak', 'ane':'aku', 'ktmu':'ketemu', 'cantiq':'cantik',
          'aq':'aku', 'netijen':'netizen', 'amvun':'ampun', 'op':'operasi', 'syekalih':'sekali','emg':'emang', 'kaya':'seperti', 'syukak':'suka', 'dongs':'dong', 'mba':'mbak',
          'lu':'kamu', 'gegara':'gara gara', 'tnpa':'tanpa', 'kl':'kalau','jng':'jangan', 'bngt':'banget', 'donk':'dong', 'bgtu':'begitu', 'kpd':'kepada',
          'kagak':'tidak', 'gak':'tidak', 'kayak':'seperti', 'cewe':'cewek', 'bacod':'bacot', 'tmbh':'tambah', 'ktm':'ketemu', 'maapkeun':'maaf', 'nya':'','pntg':'penting',
          'jan':'jangan', 'dri':'dari', 'sdh':'sudah', 'boong':'bohong', 'mehong':'mahal','gaes':'', 'couo':'cowok', 'ketjil':'kecil', 'ente':'kamu', 'qw':'aku', 'woi':'',
          'cowo':'cowok', 'manjaah':'manja','cwe':'cewek','u':'kamu', 'drpd':'daripada','manggil':'panggil', 'gt':'begitu', 'nggapapa':'tidak apa', 'tytyd':'titit', 'kaga':'tidak',
          'gtw':'tidak tahu', 'bpk':'bapak', 'w':'aku', 'fose':'pose','pokokny':'pokoknya','gw':'aku','cm':'cuma','koq':'kok','liat':'lihat','aer':'air','ngikuti':'ikut',
          'tete':'tetek','triak':'teriak','tt':'tetek','elu':'kamu','bgsd':'bangsat','gua':'aku','ntar':'nanti','embak':'mbak','mb':'mba','kasyar':'kasar','cwo':'cowok',
          'tak':'tidak','tdk':'tidak','sma':'sama','dpt':'dapat','siape':'siapa','kluar':'keluar','bgst':'bangsat','mlu':'terus','mulu':'terus','q':'aku','jlk':'jelek','guys':'',
          'krna':'karena','hrs':'harus','trs':'terus','kemaren':'kemarin','lm':'lama','dpn':'depan','sy':'saya','bg':'bang','dtg':'datang', 'laki':'lelaki','vdeo':'video','smpai':'sampai',
          'jg':'juga','ya':'iya','bnyak':'banyak','bener':'benar','loe':'kamu','kog':'kok','pengen':'ingin','elo':'kamu','knp':'kenapa','anakx':'anaknya','emak':'ibu','slu':'selalu','pastiny':'pastinya',
          'zeyenk':'sayang','kasian':'kasihan','neng':'mbak','doank':'aja', 'ati':'hati','zhyeng':'sayang','mu':'kamu','kuy':'ayo','aja':'saja','pak':'bapak','pengen':'ingin',
          'bu':'ibu','gblk':'goblok','caper':'cari perhatian','dgn':'dengan','bung':'bang','tulul':'tolol','hqq':'hakiki','g':'tidak','dy':'dia','yng':'yang','kek':'','is':'','gra':'gara',
          'ketek':'ketiak', 'yg':'yang','bkn':'bukan','ll':'', 'oplas':'operasi plastik','lalaki':'lelaki', 'woy':'','sebenernya':'sebenarnya','gabakalan':'tidak akan','bsa':'bisa','adj':'saja',
          'maen':'main','intipin':'intip', 'tercyduk':'terciduk','Uuuuuuuu':'kamu','prempun':'perempuan','yaaa':'iya','nyesal':'sesal', 'atu':'satu','aapa':'apa','pict':'gambar','pd':'pada',
          'rapetin':'rapet','bool':'pantat','ama':'sama','laen':'lain','trus':'terus', 'uda':'sudah', 'udah':'sudah','udh':'sudah','kejungkel':'jengkang','alesanya':'alasan','gue':'aku','pegel':'pegal',
          'sampe':'sampai','dlu':'dulu','byk':'banyak','melulu':'terus','bs':'bisa','jungkel':'jengkang','batangny':'batangnya','nyenggol':'senggol', 'jungkal':'jengkang','skrg':'sekarang','gasuka':'tidak suka',
          'td':'tadi','terjangkang':'jengkang','msk':'masuk','dicyduk':'ciduk','cyduk':'ciduk','eloo':'kamu','ngedj':'jd','jngan':'jangan', 'ntar':'nanti', 'gaboleh':'tidak boleh', 'yawlah':'ya allah', 'ati':'hati',
          'cwe':'cewe', 'embak':'mbak', 'pdhal':'padahal', 'mba':'mbak', 'betol':'betul', 'fto':'foto', 'perecis':'persis', 'syank':'sayang',
          'bgtu':'begitu', 'fose':'pose', 'msih':'masih', 'truz':'terus', 'uda':'udah', 'prawan':'perawan', 'kepengen':'ingin', 'filem':
          'film', 'lakik':'laki', 'indo':'indonesia', 'kliatan':'keliatan', 'puced':'pucet', 'jln':'jalan', 'atw':'atau', 'ktmuu':'ketemu',
          'amvun':'ampun', 'tercyduk':'terciduk', 'mapkeun':'maafkan', 'ntr':'nanti','mbk':'mbak', 'begitoh':'begitu', 'netijen':'netizen',
          'ngkk':'ngakak', 'ketjil':'kecil', 'TT':'tetek', 'kasian':'kasihan', 'mak':'bu', 'gt':'begitu', 'kaka':'kakak', 'nyangkang':'ngangkang',
          'mlh':'malah', 'sprti':'seperti', 'ktm':'ketemu', 'atu':'satu', 'nya':'', 'jngn':'jangan', 'dngr':'dengar', 'kt':'kata', 'orng':'orang',
          'pnting':'penting', 'tdk':'tidak', 'mrugikn':'merugikan', 'orlan':'orang lain', 'gw':'aku', 'manjaahhh':'manja', 'gua':'aku', 'mntap':
          'mantap', 'syantik':'cantik', 'pengen':'ingin', 'ajj':'aja', 'aj':'aja', 'ka':'kak', 'lbh':'lebih','kelian':'kelihatan', 'mskipun':
          'meskipun', 'ttg':'tentang', 'tw':'tau', 'aq':'aku', 'tmbh':'tambah', 'ma':'sama', 'org':'orang', 'ttp':'tetap', 'tersyuyuunkkkk':'tersayang',
          'daripd': 'daripada', 'elu':'kamu', 'pakek':'pake', 'brp':'berapa', 'dongss':'dong', 'dalm':'dalam', 'cumn':'hanya', 'cantiq':'cantik',
          'gw':'aku', 'tu':'itu', 'qaqa':'kakak', 'tabalek':'terbalik', 'motiv':'motif', 'holang':'orang', 'kl':'kalau', 'ato':'atau', 'cwo':'cowok',
          'ursin':'urusin', 'lu':'kamu', 'blm':'belum', 'jijikkkkk':'jijik', 'bru':'baru', 'na':'nya', 'teh':'itu', 'tetek':'payudara', 'luu':'kamu',
          'susu':'payudara', 'syedih':'sedih', 'bs':'bisa', 'klw':'kalau', 'tnpa':'tanpa', 'gtu':'begitu', 'ny':'nya', 'mejijikan':'menjijikan',
          'udh':'udah', 'bngt':'banget', 'doang':'saja','syngnya':'sayangnya', 'kyk':'seperti', 'brbh':'berubah', 'trus':'terus', 'ama':'sama',
          'couo':'cowok', 'hnya':'hanya', 'bkn':'bukan', 'bpk':'bapak','ga':'tidak','triak':'teriak', 'sygnya':'sayangnya', 'koment':'komentar', 'bgd':'banget',
          'laen':'lain', 'yaudh':'yaudah', 'sma':'sama', 'nti':'nanti', 'dg':'dengan', 'syekalih':'sekali', 'nich':'nih', 'kyak':'seperti',
          'ye':'ya', 'bgni':'begini', 'op':'operasi','nggak':'tidak', 'syukak':'suka', 'dsni':'disini', 'pke':'pakai', 'tamvan':'tampan'}
for index, row in df.iterrows():
    tmp = clean_word(row['Comment'], slangs=slangs)
    if(tmp==''):
        tmp = row['Comment']
    docs_clean.append(tmp)
    print(index, row['Comment'], row['Label'], tmp)

#simpan hasil preprocessing:
df['preprocessing'] = docs_clean

0 Mantap bang ente tinggi rambut gondrong cocok kalo jadi pemain basket.. 0 mantap bang kamu tinggi rambut gondrong cocok kalau jadi main basket
1 Kan kalo gini Baru cewek kalem lemah lembut gak banyak tingkah triak" gak jelas dikira orang gila 0 kan kalau begini baru cewek kalem lembut banyak teriak jelas kira orang gila
2 Lama lama benci sama lucinta dulunya suka Sekarang bikin jijik itu tetek kok gitu banget yaaa,ga ada kelihatan cewe2nyaaaa 1 lama lama benci sama lucinta suka sekarang bikin jijik payudara gitu banget iya
3 Jadi bencong kok gitu banget sih mas. 1 jadi bencong gitu banget mas
4 Aku mau mengembalikan tititku yg hilang 1 aku mau kembali titit
5 Poni tebal bgt,,kelihatan palsunya ga natural 1 poni tebal banget palsu natural
6 Yahh lo mahh emang dasarnya laki2 mau di tinggikan mau di apa juga lu tuh punya titit yg di buang 1 kamu emang dasar mau tinggi mau apa kamu punya titit buang
7 Panjat pohon ,, ehhh panjat sosial maksudnya 1 panjat panjat sosial maksud
8 Bang jakun

87 @rz_031 Intinya ya mas dia yang berbuat dia juga yang nanggung dosanya jadi ngapain harus julid haha biarlah dia yang nanggung dosa bukan anda 0 inti iya mas buat nanggung dosa jadi ngapain julid nanggung dosa bukan
88 Ya allah cantikan natural kayak gini deh 0 iya cantik natural begini
89 Cantik nya 0 cantik
90 Bang sarungnya kok model celana 0 bang sarung model celana
91 Is cantik betul natural 0 cantik betul natural
92 Nahh ini bagus foto nya natural , bagus jadi kelihatan cowo nya 1 bagus foto natural bagus jadi cowok
93 Waria semakin di depan 1 waria makin depan
94 Ratu Bulgiz 1 ratu bulgiz
95 @desiratnalexo masaaaaa. Biarkanlah urusan dia 0 masaaaaa urus
96 Itu gaya persis cwo 1 gaya persis cowok
97 @lucintaluna kaki cowo tu berbulu biar macho. 1 kaki cowok bulu biar
98 Ahhh 0 Ahhh
99 Cantik gtu ka natural...tpi syank duduknya kakak bgtu yg dibelakang sampe melihatnya bgtu 0 cantik kak natural sayang duduk kakak belakang
100 Cewek kok petakilan 1 cewek petakilan
101 Kayak bund

187 Ashgreyz dibayar brp ya buat jadi peran pendukung klo lucinta luna itu perempuan yang pasangannya laki2? Hahahhaa 1 bayar berapa iya buat jadi peran dukung kalau lucinta luna perempuan pasang
188 @nauly.bagaswati duhhh mbakkkk 0 mbakkkk
189 @rr_naldy cantikk dong 0 cantikk dong
190 Cocok nya jadi raja perkasa 1 cocok jadi raja perkasa
191 @ashgreyz suami lu tongkat nya Udah di patahin 1 suami kamu tongkat
192 @lucintaluna Dasar halu,, di dalam hatinya km menyadari kalau km itu laki2,,,krn hati gk bisa bohong,,,, cuma lo bohongin orang2,,,,dasar botak Lo.... 1 dasar kamu sadar kalau kamu cuma kamu dasar botak kamu
193 @lucintaluna iyaa solehahhh terusss mbakkk 0 iyaa terusss mbakkk
194 Ati2 Luh lucinta Luna batang 1 kamu lucinta luna batang
195 Sekilas mirip @inul.d 0 kilas mirip
196 Cantik ya sayang dia laki 1 cantik iya sayang lelaki
197 Cocoklah, harusnya emang lu jadi pembokat 1 emang kamu jadi pembokat
198 Kelihatan Cantiik yg natural 0 cantiik natural
199 Sumpah kak @lucintalu

308 Gitu dong pake sarung. Coba kalo pake peci pasti tamvan 1 gitu dong pakai sarung coba kalau pakai peci tampan
309 @ashgreyz salahhh jadi suami lahh masa istrii 1 jadi suami masa istri
310 Persis laki lo 1 persis lelaki kamu
311 @ashgreyz situ yg cocok jadii istrinya dasar ketauan boongnya klo cwo buktiin dong klo cwo 1 situ cocok jadi istri dasar tau boongnya kalau cowok buktiin dong kalau cowok
312 Emg cocok jadii pembokat 1 emang cocok jadi pembokat
313 Naturall banget kak cntik 0 naturall banget kak cantik
314 Ternyataa om fatah hahahaa 1 ternyataa om
315 Laki banget @lucintaluna 1 lelaki banget
316 ayo angkat poninya dong 0 ayo angkat poni dong
317 Laki bgt kak 1 lelaki banget kak
318 kek mas tukang parkiran 1 mas tukang parkir
319 Ada monyet transgender. 1 monyet transgender
320 Jari kaki nya cowo banget 1 jari kaki cowok banget
321 Tetep cantik 0 tetep cantik
322 Cocok bgt jadi pembantu 1 cocok banget jadi bantu
323 Cocoknya jadi topeng monyet 1 cocok jadi topeng monyet
324 t

391 @kobeydumplink jancuk 1 jancuk
392 Kira kira kalau hamil nanti keluarnya dari lubang kontol atau lubang tai ya.??? 1 kira kira kalau keluar lubang kontol lubang tai iya
393 Jangan terlalu ngeden..keluar kontol nanti. 1 jangan terlalu ngeden keluar kontol
394 punya kontol emang bisa hamil ya? 1 punya kontol emang iya
395 capek dengar kakak ngomong kayak suara burung gagak 1 capek dengar kakak ngomong suara burung gagak
396 @cmelodyc wkwkakakwkwkwk harusnya sebagai wanita ya dia kan lemah lembut atau dari gerak langkahnya kan ya memperlihatkan dia wanita dong ya kan..tapi  yaudahlah..dia blg dia wanita koq 1 wanita iya kan lembut gerak kan iya wanita dong iya kan bilang wanita kok
397 @nadhira_arza biarkan aja dia halu mau bagaimana juga..asal dia bahagia..katanya lagu dia tenar haha aku blm denger wkwkw.. males dengar nya takut kaca jendela pecah hahaa 1 biar mau bagaimana asal kata lagu tenar aku denger males dengar takut kaca jendela
398 @lestarireyna aku kesal band favorit @repvb

...

592 Lo laki laki anjay 1 kamu lelaki lelaki anjay
593 Kalau kita tau dia cowok bisa gak kalian menjaga dan tetap menghargai bisa gak kalian diam itu lebih baik daripada berbicara yang enggak2 mau bagaimanapun dia manusia sama seperti kita punya perasaan, kalau tidak bisa menghargai karena dia transgender bisa gak kalian hargai karena sesama manusia. 0 kalau tau cowok kalian jaga tetap kalian diam baik bicara mau manusia sama punya asa kalau transgender kalian sama manusia
594 Banyak kali yang lebih sexy lebih cantik di dunia ini dari lu, bangun deh bangun jangan kelamaan mimpi,lu mah ibarat cuma rontokan rengginang 1 banyak kali sexy cantik dunia kamu bangun bangun jangan lama mimpi kamu ibarat cuma rontok rengginang
595 Ternyata ratu itu juga berlaku untuk cowo 1 nyata ratu laku cowok
596 Film kayak gini ga di boikot giliran DGB di boikot. Lucu Indonesia 1 film begini boikot gilir dgb boikot lucu indonesia
597 Anjing 1 anjing
598 Lonte kelas kakap semoga insyaf mbak 1 lonte kelas kaka

678 Penonton lebih asik ngobrol di table daripada liat mbanya nge dj 1 tonton asik ngobrol table mbanya nge dj
679 @__lastribeyy__ bukan takut lagi,liat muka dj nya aja pada ogah kayaknya 1 bukan takut muka dj kayak
680 Udah ga laku 1 laku
681 Perform lebay 1 perform lebay
682 jijik gua 1 jijik aku
683 Orang nongkrong bilang "berisik banget nih musik, ga jelas pula" 1 orang nongkrong bilang berisik banget musik jelas
684 Kasihan ya.. capek2 dj tapi orang2 pada diam aja.. cuek2 aja gak ada yg goyang 1 iya dj diam goyang
685 @wulandhovanka hu uh.. Memang ga punya urat malu lagi.. AIB dibongkar, SUSU dipamerin, semuanya hanya untuk POPULARITAS & DUIT.. 1 memang punya urat malu aib bongkar payudara dipamerin semua popularitas duit
686 Kog nggak ada yg joged beb? 1 kok joged beb
687 Kayanya ga ada yg joget tamunya 1 kaya joget tamu
688 Anjir gw rasa pada takut karena yg main gunduruo 1 anjir aku rasa takut main gunduruo
689 Kok pada diem di cuekin aja tuh wkwkwkwkwk 1 diem cuekin
690 Kasian

761 Bacotttt 1 bacotttt
762 Bajingan bego bego aja anjing 1 bajing bego bego anjing
763 apasi bambang 1 apas bambang
764 Bangg udah nggak usah mikirin negara....percuma.rumah tangga aja berantakan.apa lagi mikirin negara...mau jadi apa nanti ..udah yg penting pansos..goreng aja terus kasus" yg ada yg sampai sekarang nggak tau kejelasanya dari ikan asin ikan porno ibu elsa dll...urusan negara biar jadii urusan rakyat saja.abang jangan ikut ikutan.. 1 bangg mikirin negara percuma tangga beranta apa mikirin negara mau jadi apa penting panjat sosial goreng terus kasus sekarang tau jasa ikan asin ikan porno ibu elsa urus negara biar jadi urus rakyat abang jangan ikut ikut
765 @_diego_1001 mari kita doakan 0 doa
766 @umarlahji harus di kasih kasar kata2 kalo sama farhat 1 kasar kalau sama
767 Banyak bacot 1 banyak bacot
768 Kumur2 luh hat 1 kamu
769 pansos lagi dianya liat aja followersnya naik teruss 1 panjat sosial dia followersnya naik teruss
770 Bacot puki 1 bacot puki
771 Bang di suruh 

850 Bacott 1 bacott
851 @komen_lambe ketar ketir,bacot doang gede 1 ketar tir bacot gede
852 Resep tahan dibully apa pak? 1 resep dibully apa bapak
853 Di sumpahin orang lagi..bikin orang berdosa lagi..luar biasa hidup anda 1 orang bikin orang dosa luar biasa
854 Ngomong aja lo anjing 1 ngomong kamu anjing
855 Sukses terus bang FARHAT ABBAS 1 sukses terus bang abbas
856 Pengacara hoax 1 acara
857 NGOMONG APAAN SIH LO ? KAMI CURIGANYA SAMA LO PROVOKATORNYA 1 ngomong apa kamu curiga sama kamu provokator
858 Kok ngilang ??? Di panggil polisi 1 ngilang panggil polisi
859 @dittttttttttt ahaha kerak bumi,untung bukan daki tanah 1 kerak bumi untung bukan daki
860 Klo laper tuh makan,,,, 0 kalau laper makan
861 Buktinya apa yang dibuat oleh pemerintah tidak merakyat 1 bukti apa buat rakyat
862 Ngomong apa lu bro? 1 ngomong apa kamu bro
863 Yang memilih goib suara rakyat yg coklat kali pengacara goblok 1 goib suara rakyat coklat kali acara goblok
864 Dipanggil polda km nak,sana hadapi dengan be

925 Bacot apa gak mudeng 1 bacot apa mudeng
926 Kumpulin semua janda2 lalu minta harta Gono gini.. 1 kumpulin semua lalu minta gono begini
927 @auliya_90 @enimiftahul0712 Anjir kek anak kecil lah habis buat masalah pura" dungu 1 anjir anak kecil buat pura dungu
928 Terdengar seperti kentut wkwkwwk 1 dengar kentut
929 @enimiftahul0712 bukan ketakutan lagi. Dia pengen cari bantuan ke Internasional lagi kayak nya. Ni sekarang dia mau cari perhatian pak presiden Jokowi. Dia pikir pak Jokowi peduli sama ocehan dan kata2 nya. Haha mendingan dengar suara kentut daripada dengar omongan si parah abiss ni 1 bukan takut cari bantu internasional sekarang mau cari bapak presiden jokowi pikir bapak jokowi peduli sama mending dengar suara kentut dengar omong si abiss
930 ditantang sama nyai aja ga ada nyali apa lagi om hotman aduuhh 1 tantang sama nyai nyali apa om
931 Emang lu siapa? saran lu mau di ikuti? 1 emang kamu siapa saran kamu mau ikut
932 Urusin urusan lo sm bos @hotmanparisofficial !! Ga 

In [10]:
#split text and label
Docs = df['preprocessing'].tolist()
Labels = df['Label'].tolist()
print(len(Docs), len(Labels))

for index, row in df.iterrows():
    print(index, row['preprocessing'], row['Label'])

1000 1000
0 mantap bang kamu tinggi rambut gondrong cocok kalau jadi main basket 0
1 kan kalau begini baru cewek kalem lembut banyak teriak jelas kira orang gila 0
2 lama lama benci sama lucinta suka sekarang bikin jijik payudara gitu banget iya 1
3 jadi bencong gitu banget mas 1
4 aku mau kembali titit 1
5 poni tebal banget palsu natural 1
6 kamu emang dasar mau tinggi mau apa kamu punya titit buang 1
7 panjat panjat sosial maksud 1
8 bang jakun modifikasi 1
9 Astagfirullah 0
10 muka pantat 1
11 mulai bully emang apa awal aku kakak bully pedas banget 0
12 bencong cocok dasar 1
13 mbak tetek keras batu 1
14 bencong bangga 1
15 suara kocak cowok cowok 1
16 susu kadang besar kadang kecil aku 1
17 jangan suka bully orang urusin kamu sendiri benar 0
18 jijikkkk 1
19 pasang tulang tanamm 1
20 aku dong semok montok jangan ikut 1
21 kakak cantik pinter akting jago nyanyi baik banget aku kalau ngefans sama mami depe 0
22 penting 0
23 cantik banget kalau kakak kalem begini 0
24 operasi kelamin 

262 cocok buru kalian biar jadi jangan sama sama 0
263 monyet 1
264 lelaki sekali 1
265 astagfirullah wkwkw 0
266 kalian mau cocok jadi istri suami kalian mau nama apa penting mata 0
267 natural banget 0
268 mas gaya ala lelaki 1
269 anjay kalau ngomong suka benar kamu sini 0
270 balik pantes jadi suami kamu kamu jadi istri 1
271 jangan bas abas kan cewek iya terus kalau sama om lucinta abas entot 1
272 cerita kak luna jadi apa 0
273 cantik banget makeup cewek asli kak luna semangat iya mas 1
274 seperti bakat 0
275 mirip banget sama video 0
276 cantik natural 0
277 iya cewek begini 1
278 makin jadi makin jadi 1
279 kakaku cantik natural 0
280 apa kamu sini 0
281 astaga gaya kong kuti wayer 1
282 cantik kak kalem banget 0
283 jadi suami mbak kan mbak istri 1
284 jangan lebar kali ngangkangnya 1
285 monyet 1
286 bukan orang indonesia orang 0
287 kakak cantik kalau pakai makeup 0
288 cocok sekali 0
289 orang begini puja iya bilangin orang lagu jiplak peler jiplak 1
290 jadi siapa 0
291 j

475 bacot kamu 1
476 anjing acara tv isi rusak cuma karna mau naikin rating goblok 1
477 waria 1
478 pas teriak jakun keluar bagaimana jadi 1
479 betul band repvblik tenar ketimbang baru tenar segala 1
480 wanita indonesia lucinta luna klarifikasi jiplak kelamin 1
481 lelaki laknat 1
482 iya ketua kawan monyet 1
483 bencong nongol 1
484 kan malu diri sendiri bukan si mbak pilek dulu 1
485 panjat sosial banget kamu kalau mau kenal cari cara benar bencongggggg 1
486 aku kalau ketemu kamu aku cakar salam aku kamu 1
487 titit kali lucinta luna titit balik gabisa onani 1
488 diem kamu monyet beruk 1
489 tau ngurusin kak lucinta ngapain sibuk betul mulut kalian jaga iya emang kalian jelek kak lucinta kamu apa begini kak lucinta gaya pakai duit sendiri pakai duit kalian jadi ngurus bilangin orang iya netizen sadar diri kalau mau ngomongin orang ngaca 0
490 orang undang acara tv cuma ibarat jadi aja bukan artis ibarat topeng monyet biar rame ajaaa 1
491 benar ngapain tv buat terus jelas bintan

703 sana emang dunia jadi wewe gombel umbar aurat makanya sex sama gendruwo bangga 1
704 mbak sukaknya diri sendiri mbak 1
705 serasa jual diri kalau kan trima terus siapa 1
706 pakai gilir digodain lelaki lelaki kan goblok 1
707 malu sama agama 1
708 miss genduruwo 1
709 tv panjat sosial dong 1
710 banyak banyak like benar kalian benci benar benar cinta 1
711 pikir cantik 1
712 panjat sosial 1
713 dipamerin pantes ngakunya artis youtober kamu gatelll 1
714 suka banget iya ngumbar aurat 1
715 kata mau martabat laku begini iya gunung adon donat banyak mentega 1
716 ibu anak bejat terus kata ibu anak orang baik ngga kura kura ibu ibu ngga tau sifat asli anak kalau bejat 1
717 panjat sosial dong biar undang tv buat makan 1
718 lonte burik 1
719 perkosa genderuwo 1
720 cari duit gitu banget iya 1
721 sukses orang bentar azab sama 1
722 pantas masuk neraka 1
723 jijik banget 1
724 pamer payudara kalau terima 1
725 orang kata sex sama genderuwo iya ampe rasa maaf kalau 1
726 kenal sensasi bu

971 pusing kamu sama postingan orang kotor 1
972 emang polri kpk presiden bagaimana orang jenius 1
973 jadi sekarang bukan atau bang iya fa panjat sosial terus cari trend kurang kerja kamu 1
974 cari muka 1
975 kasus video porno bang bagaimana lanjut bang 1
976 gaya iya berani panggil polda datang takut iya kamu sama bapak 1
977 kamu mati kamu korupsi nyet 1
978 begini 1
979 ngomong ngomong mau tanya kalau duit begini apa naik orang tua 1
980 depan media suci belakang begini pantas orang ngewe kamu 1
981 ingin kata kasar 1
982 jijik 1
983 sukses selalu musik keren enak dengar 1
984 telanjang 1
985 suka sula kamu 1
986 kalau balas langsung salto belakang kali mbak beb 1
987 muka tua asik asik dunia neraka imbas 1
988 iya gusti 0
989 inget wanita ibu tv pakai kerudung kamu anak susu obral telanjang sekali emang iya cari duit gitu ibu jawab kena ibu 1
990 begini ngaku kena seksual minta aku baru percaya 1
991 kenal diri orang 1
992 kampung 1
993 muka dewi sanca ratu ular iya 1
994 Murahan

In [11]:
#tfidf
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1) #min 1, klo max 100
tfidf = tfidf_vectorizer.fit_transform(Docs)

print("jumlah kolom fitur (jumlah term unik) = ",len(tfidf_vectorizer.get_feature_names()))
print(tfidf_vectorizer.get_feature_names())

matrix_tfidf = tfidf.toarray()
print(matrix_tfidf)

jumlah kolom fitur (jumlah term unik) =  1464
['aanjjiirrrrr', 'abang', 'abas', 'abbas', 'abis', 'abiss', 'acara', 'actionnya', 'ada', 'adegan', 'adil', 'adon', 'aduhhhhh', 'agama', 'agung', 'ahhh', 'aib', 'air', 'aja', 'ajaaa', 'ajak', 'ajar', 'ak', 'akibat', 'aksi', 'akting', 'aktor', 'aku', 'akun', 'ala', 'alam', 'alami', 'alay', 'album', 'alesan', 'alesannya', 'alias', 'alir', 'ambil', 'ampe', 'amplas', 'ampun', 'ampuun', 'ana', 'anak', 'andar', 'anggota', 'anggun', 'angkat', 'anjay', 'anjing', 'anjir', 'anjirr', 'anjirrr', 'anteng', 'anus', 'apa', 'apaaaaaaaaaaaa', 'apabila', 'apalgi', 'apas', 'api', 'arang', 'arti', 'artis', 'asa', 'asal', 'asap', 'asik', 'asin', 'asli', 'assalamualaikum', 'astaga', 'astagfirullah', 'astaghfirullah', 'atas', 'atau', 'atta', 'audiens', 'aura', 'auraaaat', 'aurat', 'auto', 'awal', 'awas', 'ayam', 'ayo', 'ayoo', 'ayse_ramadhani', 'azab', 'baaacccoootttt', 'baaacooott', 'babi', 'babu', 'baca', 'bacooooot', 'bacooooottt', 'bacoot', 'bacoott', 'bacoott

In [12]:
from sklearn import svm
from sklearn.model_selection import train_test_split
SVM = svm.SVC (kernel='linear')

X_train, X_test, Y_train, Y_test = train_test_split(tfidf, Labels, test_size=0.2, random_state=0)

In [13]:
import pickle

feature = {}
with open('train_tes_data.pickle', 'wb') as f:
    feature['X_train'] = X_train
    feature['X_test'] = X_test
    feature['Y_train'] = Y_train
    feature['Y_test'] = Y_test
    pickle.dump(feature, f)

In [14]:
SVM.fit(feature['X_train'] , feature['Y_train'] )

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [15]:
prediction = SVM.predict(feature['X_test'] )

print(prediction)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1
 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1
 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1]


In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(feature['Y_test'], prediction)

0.9

In [17]:
#cv = ShuffleSplit(n_splits=10,random_state=1)
#scores = cross_val_score(SVM, tfidf, Labels, cv=cv)


#for i,skor in enumerate(scores):
#    print('Pengujian ke-' + str(i+1) + ' : ' + str(scores[i]))

#print("Akurasi SVM : {}".format(scores.mean()))

In [18]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(Y_test,prediction))  
print(classification_report(Y_test,prediction))

[[ 13  14]
 [  6 167]]
              precision    recall  f1-score   support

           0       0.68      0.48      0.57        27
           1       0.92      0.97      0.94       173

    accuracy                           0.90       200
   macro avg       0.80      0.72      0.75       200
weighted avg       0.89      0.90      0.89       200



In [19]:
tn, fp, fn, tp = confusion_matrix(Y_test, prediction).ravel() #ravel untuk mengubah dari matrik 2x2 jadi 1 baris

print(tn,fp,fn,tp)

13 14 6 167


In [20]:
new_word = ['kurang ajar', 'semangat kakak', 'dasar bencong',]
tf_new_word = tfidf_vectorizer.transform(new_word)
new_predict = SVM.predict(tf_new_word)
print(new_predict)

[1 0 1]
